In [0]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import metrics
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
  data=pd.read_csv('/content/drive/My Drive/creditcard.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.5,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [0]:
fraud_data = data.loc[data['Class'] == 1]

In [0]:
fraud_data.shape
data.dtypes


Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount     object
Class       int64
dtype: object

In [0]:
data['Amount'] = pd.to_numeric(data['Amount'],errors='coerce')
data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

In [0]:
std_scaler = StandardScaler() 
rob_scaler = RobustScaler()

data['scaled_amount'] = rob_scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data['scaled_time'] = rob_scaler.fit_transform(data['Time'].values.reshape(-1,1))
data.drop(['Time','Amount'], axis=1, inplace=True)

scaled_amount = data['scaled_amount']
scaled_time = data['scaled_time']

data.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
data.insert(0, 'scaled_amount', scaled_amount)
data.insert(1, 'scaled_time', scaled_time)



data = data.sample(frac=1)

In [0]:
data.head()
data.shape

(284807, 31)

In [0]:
class Data():
    def __init__(self,data):
        self.data=data
        
    def sample(self,start,end):
        size=len(self.data)
        return self.data[int(size*start):int(size*end)]
    
    def sample_smote(self,start,end):
        data=self.sample(start,end)
        dataSmote=np.array(data.drop("Class",axis=1))
        y=np.array(data[['Class']])
        smt=SMOTE()
        dataSmote,y=smt.fit_sample(dataSmote,y)
        dataSmote=pd.DataFrame(dataSmote)
        y=pd.DataFrame(y)
        dataSmote['Class']=y
        dataSmote=dataSmote.sample(frac=1)
        dataSmote.columns=list(data.columns)
        return dataSmote

In [0]:
class Aggregator():
    
    def __init__(self):
        self.wB1=0.55
        self.wB2=0.35
        self.wB3=0.10
    
    def aggregate(self,delta,B1,B2,B3):
        delta=np.array(delta)
        temp=(self.wB1*np.array(B1) + self.wB2*np.array(B2) + self.wB3*np.array(B3))
        temp-=delta
        delta+=temp
        
        return delta
    

In [0]:
class Model():
    
    def __init__(self):    # Apply differential privacy to his model training
        self.input_shape=(30,)
        self.model = Sequential()
        self.model.add(Dense(32, activation='relu',input_shape=self.input_shape,kernel_regularizer=regularizers.l2(0.01))) 
        self.model.add(Dropout(0.2))
        self.model.add(Dense(16, activation='relu'))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(8, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(optimizer='adam',   #rmsprop
                      loss='binary_crossentropy',
                      metrics=['accuracy'])


    def run(self,X,Y,validation_split=0.0,load=True):
        self.model.fit(X,Y,epochs=1,validation_split=validation_split, verbose=1) #Validation split of 10% on the training data
        
    def evaluate(self,X,Y):
        return self.model.evaluate(X,Y)[1]*100 #Not clear what is happening here
    
    def loss(self,X,Y):
        return self.model.evaluate(X,Y)[0]
    
    def predict(self,X):
        return self.model.predict(X)

    def predictClass(self,X):
        return self.model.predict_classes(X)
        
    
    def getWeights(self):
        return self.model.get_weights()
    
    def setWeights(self,weight):
        self.model.set_weights(weight)


In [0]:
class Bank(Model):  #Bank class inherits the Model class
    
    def __init__(self,data,split_size=0):
        super().__init__()   #When we instantiate a bank, a model is automatically trained
        self.data=data
        self.split(split_size)
    
    def setData(self,data,split_size=0):
        self.data=data
        self.split(split_size)
        
    def getData(self):
        return self.data
    
    def split(self,split_size):
        X=self.data.copy()
        X.drop(['Class'],axis=1,inplace=True)
        Y=self.data[['Class']]
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(X, Y, test_size=split_size)


    

#Federated Learning


In [0]:
results={}
aggregator=Aggregator()

In [0]:
data= data.dropna()

In [0]:
data.isnull().values.any()

False

In [0]:
datum=Data(data)

Data_Test=datum.sample(0.90,1)
Data_Global=datum.sample_smote(0,0.1)              #use datum.sample  if smote not required
Data_Model_1A=datum.sample_smote(0.1,0.3)
Data_Model_2A=datum.sample_smote(0.3,0.45)
Data_Model_3A=datum.sample_smote(0.45,50)
Data_Model_1B=datum.sample_smote(0.50,70)
Data_Model_2B=datum.sample_smote(0.70,85)
Data_Model_3B=datum.sample_smote(0.85,90)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, 

In [0]:
Data_Model_1A.shape

(113738, 31)

In [0]:
GlobalBank=Bank(Data_Global,0.2) #Split the dataset into 80-20 train-test and initialize the model.
GlobalBank.run(GlobalBank.X_train, GlobalBank.Y_train,load=False) #Fit the model to train data

#Result without federated learning on global data
results['BankG.1']=GlobalBank.evaluate(GlobalBank.X_test,GlobalBank.Y_test) #Evaluate the gradient based on test data.

GlobalBank.setData(Data_Test,0.9)
yhat_classes = GlobalBank.model.predict_classes(GlobalBank.X_test)
yhat_classes = yhat_classes[:, 0]
print (yhat_classes)
recall = metrics.recall_score(GlobalBank.Y_test, yhat_classes)
f1 = metrics.f1_score(GlobalBank.Y_test, yhat_classes)
print(recall)
print(f1)


Train on 40947 samples, validate on 4550 samples
Epoch 1/1
11375/11375 [==============================] - 0s 19us/step
[0 0 0 ... 0 0 0]
0.7948717948717948
0.5688073394495412


In [0]:
GlobalBank.setData(Data_Global,0.2)


In [0]:

#ITERATION 1 OF THE FDERATED LEARNING FRAMEWORK

Bank1=Bank(Data_Model_1A, 0.2)
Bank1.run(Bank1.X_train,Bank1.Y_train)

Bank2=Bank(Data_Model_2A, 0.2)
Bank2.run(Bank2.X_train,Bank2.Y_train)

Bank3=Bank(Data_Model_3A, 0.2)
Bank3.run(Bank3.X_train,Bank3.Y_train)

delta=aggregator.aggregate(GlobalBank.getWeights(),Bank1.getWeights(),Bank2.getWeights(),Bank3.getWeights())

GlobalBank.setWeights(delta)

GlobalBank.setData(Data_Test,0.9) 
yhat_classes = GlobalBank.model.predict_classes(GlobalBank.X_test)
yhat_classes = yhat_classes[:, 0]
recall = metrics.recall_score(GlobalBank.Y_test, yhat_classes)
f1 = metrics.f1_score(GlobalBank.Y_test, yhat_classes)
print(recall)
print(f1)
precision = (f1*recall)/((2*recall) - f1)
print (precision)

Train on 81891 samples, validate on 9099 samples
Epoch 1/1
81891/81891 [==============================] - 4s 48us/step - loss: 0.1259 - accuracy: 0.9766 - val_loss: 0.0312 - val_accuracy: 0.9956
Train on 61402 samples, validate on 6823 samples
Epoch 1/1
61402/61402 [==============================] - 3s 47us/step - loss: 0.1239 - accuracy: 0.9770 - val_loss: 0.0250 - val_accuracy: 0.9987
Train on 225169 samples, validate on 25019 samples
Epoch 1/1
225169/225169 [==============================] - 11s 47us/step - loss: 0.0845 - accuracy: 0.9813 - val_loss: 0.0364 - val_accuracy: 0.9963
0.9761904761904762
0.0035560952339650466
0.0017812920884563586


In [0]:
GlobalBank.setData(Data_Global,0.2)

In [0]:
results['Bank1.1']=Bank1.evaluate(Bank1.X_test,Bank1.Y_test)
results['Bank2.1']=Bank2.evaluate(Bank2.X_test,Bank2.Y_test)
results['Bank3.1']=Bank3.evaluate(Bank3.X_test,Bank3.Y_test)
results['BankG.2']=GlobalBank.evaluate(GlobalBank.X_test,GlobalBank.Y_test) #Result after the 1st iteration of federated learning

11375/11375 [==============================] - 0s 19us/step


In [0]:
#ITERATION 2 OF FEDERATED LEARNING

Bank1.setData(Data_Model_1B,0.2)
Bank1.run(Bank1.X_train,Bank1.Y_train)

Bank2.setData(Data_Model_2B,0.2)
Bank2.run(Bank2.X_train,Bank2.Y_train)

Bank3.setData(Data_Model_3B,0.2)
Bank3.run(Bank3.X_train,Bank3.Y_train)



delta=aggregator.aggregate(GlobalBank.getWeights(),Bank1.getWeights(),Bank2.getWeights(),Bank3.getWeights())

GlobalBank.setWeights(delta)

GlobalBank.setData(Data_Test,0.9) 
yhat_classes = GlobalBank.model.predict_classes(GlobalBank.X_test)
yhat_classes = yhat_classes[:, 0]
recall = metrics.recall_score(GlobalBank.Y_test, yhat_classes)
f1 = metrics.f1_score(GlobalBank.Y_test, yhat_classes)
print(recall)
print(f1)
precision = (f1*recall)/((2*recall) - f1)
print (precision)

Train on 204694 samples, validate on 22744 samples
Epoch 1/1
204694/204694 [==============================] - 9s 45us/step - loss: 0.0532 - accuracy: 0.9879 - val_loss: 0.0365 - val_accuracy: 0.9944
Train on 122806 samples, validate on 13646 samples
Epoch 1/1
122806/122806 [==============================] - 6s 46us/step - loss: 0.0405 - accuracy: 0.9922 - val_loss: 0.0242 - val_accuracy: 0.9976
Train on 61404 samples, validate on 6823 samples
Epoch 1/1
61404/61404 [==============================] - 3s 46us/step - loss: 0.0271 - accuracy: 0.9975 - val_loss: 0.0198 - val_accuracy: 0.9991
1.0
0.09603340292275572
0.05043859649122806


In [0]:
GlobalBank.setData(Data_Global,0.2)

In [0]:
results['Bank1.2']=Bank1.evaluate(Bank1.X_test,Bank1.Y_test)
results['Bank2.2']=Bank2.evaluate(Bank2.X_test,Bank2.Y_test)
results['Bank3.2']=Bank3.evaluate(Bank3.X_test,Bank3.Y_test)


17057/17057 [==============================] - 0s 19us/step


In [0]:
results

{'Bank1.1': 99.529629945755,
 'Bank1.2': 99.36862587928772,
 'Bank2.1': 99.84756708145142,
 'Bank2.2': 99.66875910758972,
 'Bank3.1': 99.67225193977356,
 'Bank3.2': 99.91205930709839,
 'BankG.1': 99.56923127174377,
 'BankG.2': 54.93626594543457}

In [0]:
#Our final model M'' is now in GlobalBank
GlobalBank.setData(Data_Test,0.9)
results['BankG.3']=GlobalBank.evaluate(GlobalBank.X_test,GlobalBank.Y_test)

25633/25633 [==============================] - 0s 18us/step


In [0]:
results

{'Bank1.1': 99.529629945755,
 'Bank1.2': 99.36862587928772,
 'Bank2.1': 99.84756708145142,
 'Bank2.2': 99.66875910758972,
 'Bank3.1': 99.67225193977356,
 'Bank3.2': 99.91205930709839,
 'BankG.1': 99.56923127174377,
 'BankG.2': 54.93626594543457,
 'BankG.3': 96.6839611530304}

In [0]:
ynew = GlobalBank.predict(GlobalBank.X_test)

0


In [0]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.5,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [0]:
df = pd.DataFrame(dict) 

In [0]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.5516,-0.617801,-0.99139,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.5516,-0.617801,-0.99139,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0


In [0]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [0]:
GlobalBank=Bank(data,0.2) #Split the dataset into 80-20 train-test and initialize the model.
GlobalBank.run(GlobalBank.X_train, GlobalBank.Y_train,load=False) #Fit the model to train data

#Result without federated learning on global data
results['BankG.1']=GlobalBank.evaluate(GlobalBank.X_test,GlobalBank.Y_test) #Evaluate the gradient based on test data.


Epoch 1/1
56962/56962 [==============================] - 1s 21us/step


In [0]:

for row in data.iterrows():
  dict = {
  'Time':[row[1]['Time'], row[1]['Time']], 
  'V1': [row[1]['V1'], row[1]['V1']], 
  'V2': [row[1]['V2'], row[1]['V2']],
  'V3': [row[1]['V3'], row[1]['V3']],
  'V4': [row[1]['V4'], row[1]['V4']],
  'V5': [row[1]['V5'], row[1]['V5']],
  'V6': [row[1]['V6'], row[1]['V6']],
  'V7': [row[1]['V7'], row[1]['V7']],
  'V8': [row[1]['V8'], row[1]['V8']],
  'V9': [row[1]['V9'], row[1]['V9']],
  'V10': [row[1]['V10'], row[1]['V10']],
  'V11': [row[1]['V11'], row[1]['V11']],
  'V12': [row[1]['V12'],row[1]['V12']],
  'V13': [row[1]['V13'], row[1]['V13']],
  'V14': [row[1]['V14'], row[1]['V14']],
  'V15': [row[1]['V15'], row[1]['V15']],
  'V16': [row[1]['V16'], row[1]['V16']],
  'V17': [row[1]['V17'], row[1]['V17']],
  'V18': [row[1]['V18'], row[1]['V18']],
  'V19': [row[1]['V19'], row[1]['V19']],
  'V20': [row[1]['V20'], row[1]['V20']],
  'V21': [row[1]['V21'], row[1]['V21']],
  'V22': [row[1]['V22'], row[1]['V22']],
  'V23': [row[1]['V23'], row[1]['V23']],
  'V24': [row[1]['V24'], row[1]['V24']],
  'V25': [row[1]['V25'], row[1]['V25']],
  'V26': [row[1]['V26'], row[1]['V26']],
  'V27': [row[1]['V27'], row[1]['V27']],
  'V28': [row[1]['V28'], row[1]['V28']],
  'Amount':[row[1]['Amount'], row[1]['Amount']],
  'Class' : [row[1]['Class'], row[1]['Class']]} 


  df = pd.DataFrame(dict) 
  #1 ITERATION OF THE FDERATED LEARNING FRAMEWORK
  Bank1=Bank(df, 0.5)
  Bank1.run(Bank1.X_train,Bank1.Y_train)

  Bank2=Bank(df, 0.5)
  Bank2.run(Bank2.X_train,Bank2.Y_train)

  Bank3=Bank(df, 0.5)
  Bank3.run(Bank3.X_train,Bank3.Y_train)

  delta=aggregator.aggregate(GlobalBank.getWeights(),Bank1.getWeights(),Bank2.getWeights(),Bank3.getWeights())

  GlobalBank.setWeights(delta)


Epoch 1/1
1/1 [==============================] - 0s 166ms/step - loss: 0.3190 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 176ms/step - loss: 2.4127 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 172ms/step - loss: 24.9610 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 180ms/step - loss: 0.7974 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 178ms/step - loss: 1.1279 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 172ms/step - loss: 1.1192 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 178ms/step - loss: 18.1080 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 213ms/step - loss: 34.2861 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 233ms/step - loss: 3.7315 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 193ms/step - loss: 10.0412 - accuracy: 0.000